In [1]:
!git clone https://github.com/Francesco9932/rag_risiko

Cloning into 'rag_risiko'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 5), reused 8 (delta 2), pack-reused 0
Receiving objects: 100% (14/14), 1.80 MiB | 8.93 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

!pip install pypdf
!pip install spacy
!python3 -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [3]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [4]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("./rag_risiko/Regolamento-Risiko.pdf")
pages = loader.load()

In [5]:
pages[1].page_content

'Il gioco strategico\nper la conquista\ndel mondo\nDa 3 a 6 giocatori\nEtà: da 10 anni in su\n2\n'

In [6]:
from langchain.text_splitter import SpacyTextSplitter
# spacy text splitter
splitter = SpacyTextSplitter(pipeline="it_core_news_sm")
docs = splitter.split_documents(pages)

In [7]:
docs[2]

Document(page_content='3SCOPO DEL GIOCO\n\n\nRaggiungere per primi il proprio obiettivo segreto.\nCOMPONENTI\nUn piano di gioco, rappresentante un planisfero suddiviso \nin 42 territori, appartenenti ai 6 continenti.\n\nSu ogni \nterritorio è indicato un Punteggio Vittoria.\n\nTale valore \nserve esclusivamente se decidete di utilizzare le nuove \nregole per il gioco a tempo ridotto.\n\n\n6 armate di diverso colore composte da piccoli carrarmati \n(del valore di un’armata ciascuno) e bandierine (del valore \ndi 10 armate) 3 dadi di colore rosso e 3 dadi di colore blu.\n\n\nUn mazzo di carte “Territori” così suddiviso:\n\n\n\n\n•\t42\tcarte\traffiguranti\t ciascuna\t uno\tdei\tterritori \n presenti sulla plancia di gioco e il simbolo di una \n delle tre armi dell’esercito (fante, cavallo, cannone).\n\n\n•\t2\tcarte\t“Jolly”\tdistinguibili\t dalle\taltre\tper\tla\tpresenza \n dei simboli di tutte le armi e l’assenza della mappa.\n\n\nUn mazzo di carte “Obiettivi”, indicanti gli obiettivi

In [8]:
# Define the path to the pre-trained model you want to use
modelPath = "nickprock/sentence-bert-base-italian-uncased"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/732k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
db = FAISS.from_documents(docs, embeddings)

In [16]:
question = "numero di settori"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

3SCOPO DEL GIOCO


Raggiungere per primi il proprio obiettivo segreto.
COMPONENTI
Un piano di gioco, rappresentante un planisfero suddiviso 
in 42 territori, appartenenti ai 6 continenti.

Su ogni 
territorio è indicato un Punteggio Vittoria.

Tale valore 
serve esclusivamente se decidete di utilizzare le nuove 
regole per il gioco a tempo ridotto.


6 armate di diverso colore composte da piccoli carrarmati 
(del valore di un’armata ciascuno) e bandierine (del valore 
di 10 armate) 3 dadi di colore rosso e 3 dadi di colore blu.


Un mazzo di carte “Territori” così suddiviso:




•	42	carte	raffiguranti	 ciascuna	 uno	dei	territori 
 presenti sulla plancia di gioco e il simbolo di una 
 delle tre armi dell’esercito (fante, cavallo, cannone).


•	2	carte	“Jolly”	distinguibili	 dalle	altre	per	la	presenza 
 dei simboli di tutte le armi e l’assenza della mappa.


Un mazzo di carte “Obiettivi”, indicanti gli obiettivi segreti 
che i giocatori dovranno tentare di raggiungere.


In [17]:
# Questa parte da rivedere:

# Specify the model name you want to use
model_name = "Intel/dynamic_tinybert"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.0, "max_length": 512},
)

In [18]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [19]:
question = "qual'è il numero massimo di giocatori?"
result = qa.run({"query": question})
print(result["result"])

ValueError: Context information is below. 
------------
FINALE
DI PARTITA
E MODALITÀ DI GIOCO TIME ATTACK
Prima di iniziare a giocare dovete decidere se volete finire	 la	
partita nel modo “tradizionale” oppure se volete giocare il 
metodo	“TIME	ATTACK”.


Se decidete di giocare nel modo tradizionale, la partita 
proseguirà	 fino	a	quando	 uno	dei	giocatori	 non	avrà	
raggiunto l’obiettivo indicato sulla carta “obiettivo segreto” 
pescata all’inizio del gioco.


Se	invece	decidete	 di	utilizzare	 il	metodo	 “TIME	ATTACK” 	
dovete anche decidere se volete giocare con un limite di tempo 
(comunque allo scadere del tempo bisogna completare il giro 
in modo che tutti abbiano potuto giocare lo stesso numero di turni) oppure sulla base delle carte giocate.


Durante	 la	partita,	 le	carte	giocate	 per	ottenere	 rinforzi	
vengono	 poste	nel	mazzo	degli	scarti.	

Quando	 il	mazzo	
“territori” si esaurisce, il giocatore che ha pescato l’ultima 
carta dovrà mescolare tutte le carte del mazzo deg